In [ ]:
# !pip install natasha

In [2]:
from natasha import (
Segmenter,
MorphVocab,

NewsEmbedding,
NewsMorphTagger,
NewsSyntaxParser,
NewsNERTagger, 

PER,
AddrExtractor,

Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = AddrExtractor(morph_vocab)

In [ ]:
from slovnet import NER
from navec import Navec
from tqdm import tqdm
import os
import csv
import re

# text = 'Европейский союз добавил в санкционный список девять политических деятелей из самопровозглашенных республик Донбасса — Донецкой народной республики (ДНР) и Луганской народной республики (ЛНР) — в связи с прошедшими там выборами. Об этом говорится в документе, опубликованном в официальном журнале Евросоюза. В новом списке фигурирует Леонид Пасечник, который по итогам выборов стал главой ЛНР. Помимо него там присутствуют Владимир Бидевка и Денис Мирошниченко, председатели законодательных органов ДНР и ЛНР, а также Ольга Позднякова и Елена Кравченко, председатели ЦИК обеих республик. Выборы прошли в непризнанных республиках Донбасса 11 ноября. На них удержали лидерство действующие руководители и партии — Денис Пушилин и «Донецкая республика» в ДНР и Леонид Пасечник с движением «Мир Луганщине» в ЛНР. Президент Франции Эмманюэль Макрон и канцлер ФРГ Ангела Меркель после встречи с украинским лидером Петром Порошенко осудили проведение выборов, заявив, что они нелегитимны и «подрывают территориальную целостность и суверенитет Украины». Позже к осуждению присоединились США с обещаниями новых санкций для России.'

navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)

In [ ]:
def get_names_by_slovnet(text):
    return [span for span in ner(text).spans if span.type == 'PER']


def get_names_by_natasha(doc):
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)

    names = []
    for span in doc.spans:
        if span.type == PER:
            names.append(span)
    return names

In [ ]:
import os
from tqdm import tqdm

corpus_dir = './corpus_for_diploma'

filenames_names = {}
for filename in tqdm(os.listdir(corpus_dir)):
    with open(os.path.join(corpus_dir, filename), "r") as f:
        filenames_names[filename] = get_names_by_slovnet(
            re.sub(' +', ' ', re.sub('{\d+?}', '', f.read()))
        )

import json


with open('ner_result.json', 'w') as f:
    json.dump(filenames_names, f)

# with open('ner_result.json', 'r') as f:
#     filenames_names = json.load(f)

for filename in tqdm(os.listdir(corpus_dir)):
    with open(os.path.join(corpus_dir, filename), "r") as f:
#         text = f.read()
        text = re.sub(' +', ' ', re.sub('{\d+?}', '', f.read()))
        for i, span in enumerate(filenames_names[filename]):
            filenames_names[filename][i]['text'] = text[span['start']:span['stop']]

   # Фильтруем имена (оставляем только с фамилией)

In [132]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()


def is_with_surn(name_text):
    for name_part in name_text.split():
        normal_form = morph.parse(name_part)[0].normal_form
        for form in morph.parse(normal_form):
            if 'Surn' in form.tag:
                return True
    return False

def get_surn(name_text):
    for name_part in name_text.split():
        normal_form = morph.parse(name_part)[0].normal_form
        for form in morph.parse(normal_form):
            if 'Surn' in form.tag:
                return form.normal_form
    return False

In [146]:
only_with_fio = {}

for filename in tqdm(os.listdir(corpus_dir)):
    only_with_fio[filename] = []
    for i, span in enumerate(filenames_names[filename]):
        if is_with_surn(span['text']):
            only_with_fio[filename].append(span)

100%|██████████| 506/506 [00:21<00:00, 23.94it/s]


In [ ]:
with open('only_with_fio.json', 'w') as f:
    json.dump(only_with_fio, f, ensure_ascii=False, indent=3)

## Достаем предложения, содержащие упоминания персоналий

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer as pt


def get_sentences_with_idx(book_text):
    spans = list(pt().span_tokenize(book_text))
    result = [
        {
            'start': span[0],
            'stop': span[1],
            'sen_text': book_text[span[0]:span[1]]
        }
        for span in spans
    ]
    return result

In [ ]:
filenames_sents = {}

for filename in tqdm(os.listdir(corpus_dir)):
    with open(os.path.join(corpus_dir, filename), "r") as f:
        text = re.sub(' +', ' ', re.sub('{\d+?}', '', f.read()))
        filenames_sents[filename] = get_sentences_with_idx(text)

In [ ]:
def get_sent_by_idx(span, text_sents):
    for sent in text_sents:
        if sent['start'] <= span['start'] and sent['stop'] >= span['stop']:
            return sent

In [ ]:
with_sents = {}
for filename in tqdm(only_with_fio):
    with_sents[filename] = []
    for i, span in enumerate(only_with_fio[filename]):
        sent = get_sent_by_idx(span, filenames_sents[filename])
        if sent:
            span.update({'sentence': sent})
            with_sents[filename].append(span)

In [ ]:
for filename in tqdm(with_sents):
    for i, span in enumerate(with_sents[filename]):
        with_sents[filename][i]['norm_surn'] = get_surn(span['text'])

In [ ]:
with open('with_sents.json', 'w') as f:
    json.dump(with_sents, f, ensure_ascii=False, indent=3)

--- 

In [31]:
import json
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

with open('with_sents.json', 'r') as f:
    with_sents = json.load(f)

In [26]:
books_entities = []
for book in with_sents:
    book_entities_ = pd.DataFrame(with_sents[book])
    book_entities_.sentence = book_entities_.sentence.apply(lambda sentence: sentence['sen_text'])
    book_entities_ = book_entities_[['norm_surn', 'sentence']].copy()
    book_entities_['book'] = book
    book_entities_['author'] = book.split('.')[0]  
    books_entities.append(book_entities_)
book_entities = pd.concat(books_entities)
book_entities

,norm_surn,sentence,book,author
0,павлищев,"-- Да, господин Павлищев, который меня там сод...",dostoyevskiy.idiot.txt,dostoyevskiy
1,епанчин,"-- Да, господин Павлищев, который меня там сод...",dostoyevskiy.idiot.txt,dostoyevskiy
2,епанчин,-- Узелок ваш все-таки имеет некоторое значени...,dostoyevskiy.idiot.txt,dostoyevskiy
3,епанчин,-- Узелок ваш все-таки имеет некоторое значени...,dostoyevskiy.idiot.txt,dostoyevskiy
4,павлищев,"Гм... генерала же Епанчина знаем-с, собственно...",dostoyevskiy.idiot.txt,dostoyevskiy
...,...,...,...,...
55,жук,Жук поместился в ногах.,maklakova_nelidova.devochka_lida.txt,maklakova_nelidova
56,дмитриев,Она крепко прижалась к Дмитриевой поддевке.,maklakova_nelidova.devochka_lida.txt,maklakova_nelidova
57,жук,Жука некуда было взять; тетя ни за что не позв...,maklakova_nelidova.devochka_lida.txt,maklakova_nelidova
58,жук,"Бедный Жук лаял, бегал вокруг коляски; Аксюша ...",maklakova_nelidova.devochka_lida.txt,maklakova_nelidova


In [ ]:
norm_surns = book_entities.norm_surn.unique()
len(norm_surns)

# Сортируем вымышленных от невымышленных 

In [64]:
import wikipedia

wikipedia.set_lang('ru')

In [110]:
def get_status(name, try_idx_limit=1):
    search_results = wikipedia.search(name)
    if search_results:
        try_idx = 0
        page = None
        while page is None:
            try:
                found_name = search_results[try_idx]
                try:
                    page = wikipedia.page(found_name)
                except wikipedia.DisambiguationError as e:
                    s = random.choice(e.options)
                    page = wikipedia.page(s)
            except:
                try_idx += 1
                if try_idx == try_idx_limit:
                    return None, None, None

        birth_year = None
        for category in page.categories:
            if category.startswith('Категория:Родившиеся в ') and category.endswith('году'):
                birth_year = int(category[23:27])
        return found_name, 'Категория:Персоналии по алфавиту' in page.categories, birth_year
    else:
        return None, None, None

In [66]:
def filter_names(names):
    filtered_names = []
    for name in tqdm(names):
        found_name, status, birth_year = get_status(name)
        if status and birth_year is not None and birth_year < 1890:
            filtered_names.append([found_name, status, birth_year])
    return filtered_names

In [105]:
filtered_names = {}

In [106]:
for name in tqdm(norm_surns):
    found_name, status, birth_year = get_status(name)
    if status and birth_year is not None and birth_year < 1880:
        filtered_names[name] = [found_name, status, birth_year]

100%|██████████| 229/229 [07:28<00:00,  1.96s/it]


# Анализ тональности

In [46]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment', return_dict=True)
model.to('cuda')

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    inputs.to('cuda')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1).detach().cpu()#.numpy()
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

In [55]:
book_entities['sentiment_class'] = book_entities.sentence.progress_apply(predict)

100%|██████████| 202146/202146 [25:39<00:00, 131.29it/s]


In [56]:
book_entities.sentiment_class.value_counts()

0    184166
1     10109
2      7871
Name: sentiment_class, dtype: int64

# Добавляем метаданные

In [60]:
import pandas as pd


metadata_df = pd.read_csv('metadata.tsv', sep='\t')
metadata_dict = metadata_df[['filename', 'author', 'title', 'year']].to_dict('records')
metadata_dict = {r['filename']: r for r in metadata_dict}
metadata_dict

{'akhsharumov.kontsy_v_vodu.txt': {'filename': 'akhsharumov.kontsy_v_vodu.txt',
  'author': 'Ахшарумов, Николай Дмитриевич',
  'title': 'Концы в воду',
  'year': 1872},
 'aksakov.detskiye_gody_bagrova_vnuka.txt': {'filename': 'aksakov.detskiye_gody_bagrova_vnuka.txt',
  'author': 'Аксаков, Сергей Тимофеевич',
  'title': 'Детские годы Багрова-внука',
  'year': 1858},
 'aksakov.semeynaya_khronika.txt': {'filename': 'aksakov.semeynaya_khronika.txt',
  'author': 'Аксаков, Сергей Тимофеевич',
  'title': 'Семейная хроника',
  'year': 1856},
 'alekseyev.igra_sudby.txt': {'filename': 'alekseyev.igra_sudby.txt',
  'author': 'Алексеев, Николай Николаевич',
  'title': 'Игра судьбы',
  'year': 1913},
 'alekseyev.lzhetsarevich.txt': {'filename': 'alekseyev.lzhetsarevich.txt',
  'author': 'Алексеев, Николай Николаевич',
  'title': 'Лжецаревич',
  'year': 1898},
 'amfiteatrov.otravlennaya_sovest.txt': {'filename': 'amfiteatrov.otravlennaya_sovest.txt',
  'author': 'Амфитеатров, Александр Валентинович

In [61]:
book_entities['pretty_author'] = book_entities.book.apply(lambda book: metadata_dict[book]['author'])
book_entities['pretty_title'] = book_entities.book.apply(lambda book: metadata_dict[book]['title'])
book_entities['pretty_year'] = book_entities.book.apply(lambda book: metadata_dict[book]['year'])
book_entities

,norm_surn,sentence,book,author,sentiment_class,pretty_author,pretty_title,pretty_year
0,павлищев,"-- Да, господин Павлищев, который меня там сод...",dostoyevskiy.idiot.txt,dostoyevskiy,0,"Достоевский, Фёдор Михайлович",Идиот,1874
1,епанчин,"-- Да, господин Павлищев, который меня там сод...",dostoyevskiy.idiot.txt,dostoyevskiy,0,"Достоевский, Фёдор Михайлович",Идиот,1874
2,епанчин,-- Узелок ваш все-таки имеет некоторое значени...,dostoyevskiy.idiot.txt,dostoyevskiy,0,"Достоевский, Фёдор Михайлович",Идиот,1874
3,епанчин,-- Узелок ваш все-таки имеет некоторое значени...,dostoyevskiy.idiot.txt,dostoyevskiy,0,"Достоевский, Фёдор Михайлович",Идиот,1874
4,павлищев,"Гм... генерала же Епанчина знаем-с, собственно...",dostoyevskiy.idiot.txt,dostoyevskiy,0,"Достоевский, Фёдор Михайлович",Идиот,1874
...,...,...,...,...,...,...,...,...
55,жук,Жук поместился в ногах.,maklakova_nelidova.devochka_lida.txt,maklakova_nelidova,0,"Маклакова (Королёва), Лидия Филипповна",Девочка Лида,1875
56,дмитриев,Она крепко прижалась к Дмитриевой поддевке.,maklakova_nelidova.devochka_lida.txt,maklakova_nelidova,0,"Маклакова (Королёва), Лидия Филипповна",Девочка Лида,1875
57,жук,Жука некуда было взять; тетя ни за что не позв...,maklakova_nelidova.devochka_lida.txt,maklakova_nelidova,0,"Маклакова (Королёва), Лидия Филипповна",Девочка Лида,1875
58,жук,"Бедный Жук лаял, бегал вокруг коляски; Аксюша ...",maklakova_nelidova.devochka_lida.txt,maklakova_nelidova,0,"Маклакова (Королёва), Лидия Филипповна",Девочка Лида,1875
